In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [5]:
# 데이터 로드 및 전처리
data = pd.read_csv('abalone.csv')
data.columns = ['Sex', 'Length', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight', 
                'VisceraWeight', 'ShellWeight', 'Rings']

In [15]:
# 성별 원-핫 인코딩
encoder = OneHotEncoder(sparse_output=False)
sex_encoded = encoder.fit_transform(data[['Sex']])

# 피처와 라벨 분리
features = np.concatenate([sex_encoded, data.iloc[:, 1:-1].values], axis=1)
labels = data['Rings'].values.reshape(-1, 1)  # 텐서플로와 호환되도록 차원 추가

# 정규화
scaler = StandardScaler()
features = scaler.fit_transform(features)

# 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# 입력 데이터를 float32로 변환
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# 입력 데이터 및 출력 데이터의 크기
input_dim = x_train.shape[1]

In [16]:
# 모델 정의
class AbaloneModel(tf.Module):
    def __init__(self):
        self.w1 = tf.Variable(tf.random.normal([input_dim, 64], stddev=0.03, dtype=tf.float32), name='w1')
        self.b1 = tf.Variable(tf.zeros([64], dtype=tf.float32), name='b1')
        self.w2 = tf.Variable(tf.random.normal([64, 32], stddev=0.03, dtype=tf.float32), name='w2')
        self.b2 = tf.Variable(tf.zeros([32], dtype=tf.float32), name='b2')
        self.w3 = tf.Variable(tf.random.normal([32, 1], stddev=0.03, dtype=tf.float32), name='w3')
        self.b3 = tf.Variable(tf.zeros([1], dtype=tf.float32), name='b3')


    def __call__(self, x):
        h1 = tf.nn.relu(tf.matmul(x, self.w1) + self.b1)
        h2 = tf.nn.relu(tf.matmul(h1, self.w2) + self.b2)
        return tf.matmul(h2, self.w3) + self.b3

# 모델 인스턴스 생성
model = AbaloneModel()

# 손실 함수 정의 (Mean Squared Error)
def mse_loss(y_pred, y_true):
    return tf.reduce_mean(tf.square(y_pred - y_true))

# 옵티마이저 정의
optimizer = tf.optimizers.Adam(learning_rate=0.001)

In [17]:
# 훈련 루프
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        predictions = model(x_batch)
        loss = mse_loss(predictions, y_batch)
    gradients = tape.gradient(loss, [model.w1, model.b1, model.w2, model.b2, model.w3, model.b3])
    optimizer.apply_gradients(zip(gradients, [model.w1, model.b1, model.w2, model.b2, model.w3, model.b3]))
    return loss

In [18]:
# 데이터셋 생성 (미니배치 처리)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [19]:
# 모델 학습
epochs = 50
for epoch in range(epochs):
    epoch_loss = 0.0
    for x_batch, y_batch in train_dataset:
        batch_loss = train_step(x_batch, y_batch)
        epoch_loss += batch_loss.numpy()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(x_train):.4f}")

Epoch 1/50, Loss: 2.3139
Epoch 2/50, Loss: 0.6036
Epoch 3/50, Loss: 0.2389
Epoch 4/50, Loss: 0.1794
Epoch 5/50, Loss: 0.1647
Epoch 6/50, Loss: 0.1597
Epoch 7/50, Loss: 0.1569
Epoch 8/50, Loss: 0.1547
Epoch 9/50, Loss: 0.1517
Epoch 10/50, Loss: 0.1510
Epoch 11/50, Loss: 0.1522
Epoch 12/50, Loss: 0.1494
Epoch 13/50, Loss: 0.1482
Epoch 14/50, Loss: 0.1479
Epoch 15/50, Loss: 0.1476
Epoch 16/50, Loss: 0.1465
Epoch 17/50, Loss: 0.1472
Epoch 18/50, Loss: 0.1469
Epoch 19/50, Loss: 0.1456
Epoch 20/50, Loss: 0.1451
Epoch 21/50, Loss: 0.1444
Epoch 22/50, Loss: 0.1443
Epoch 23/50, Loss: 0.1439
Epoch 24/50, Loss: 0.1443
Epoch 25/50, Loss: 0.1431
Epoch 26/50, Loss: 0.1427
Epoch 27/50, Loss: 0.1426
Epoch 28/50, Loss: 0.1426
Epoch 29/50, Loss: 0.1433
Epoch 30/50, Loss: 0.1417
Epoch 31/50, Loss: 0.1415
Epoch 32/50, Loss: 0.1410
Epoch 33/50, Loss: 0.1440
Epoch 34/50, Loss: 0.1413
Epoch 35/50, Loss: 0.1405
Epoch 36/50, Loss: 0.1409
Epoch 37/50, Loss: 0.1402
Epoch 38/50, Loss: 0.1401
Epoch 39/50, Loss: 0.

In [20]:
# 평가
def evaluate(dataset):
    total_loss = 0.0
    for x_batch, y_batch in dataset:
        predictions = model(x_batch)
        loss = mse_loss(predictions, y_batch)
        total_loss += loss.numpy()
    return total_loss / len(x_test)

test_loss = evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}")

Test Loss: 0.1415
